In [39]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [60]:
df = pd.read_csv('../DataSets/mushrooms.csv')
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [61]:
le = LabelEncoder()
df = df.apply( le.fit_transform )
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [62]:
split = int( 0.9*df.values.shape[0] )
print split
train = df.loc[ :split ]
test = df.loc[ split: ]
print train.shape
print test.shape
test = test.reset_index().drop( 'index', axis = 1 )
test.head()

7311
(7312, 23)
(813, 23)


,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,0,5,0,8,0,5,1,1,0,2,...,2,7,7,0,2,2,4,7,3,1
1,1,3,3,2,0,8,1,0,1,0,...,2,7,6,0,2,1,0,7,4,2
2,1,3,3,4,0,7,1,0,1,0,...,1,7,7,0,2,1,0,7,4,4
3,0,5,2,3,0,5,1,1,0,7,...,2,7,7,0,2,2,4,7,2,1
4,0,5,2,8,0,5,1,1,0,2,...,1,7,7,0,2,2,4,7,2,1


In [63]:
print type( df['type'] )

<class 'pandas.core.series.Series'>


In [64]:
def prior_prob ( data, label_column, label_value ):
    n_rows = data.loc[ data[label_column] == label_value ]
    #print n_rows.shape
    return n_rows.shape[0]/float( data.shape[0] )

print prior_prob( train, 'type', 1 )

0.473605032823


In [65]:
def cond_prob ( data, feature_column, feature_value, label_column, label_value ):
    constraint = data.loc[ ( data[label_column] == label_value ) & ( data[feature_column] == feature_value ) ]
    #print constraint.shape
    n_rows = data.loc[ data[label_column] == label_value ]
    return constraint.shape[0]/float( n_rows.shape[0] )

print cond_prob( train, 'cap_shape', 5, 'type', 0 )

0.476747207067


# Naive Bayes Implementation Loop

In [69]:
preds = []
CLASSES = np.unique( test['type'] )
for ix in range( test.shape[0] ):
    val = test.loc[ ix ][1:]
    features = dict( val )
    posterior = []
    for cx in CLASSES:
        likelihood = 1.0
        for fx in features:
            cp = cond_prob( train, fx, features[fx], 'type', cx )
            likelihood *= cp
        
        prior = prior_prob( train, 'type', cx )
        posterior_prob = prior*likelihood
        posterior.append( posterior_prob )
    
    prediction = np.array( posterior ).argmax()
    preds.append( prediction )

In [67]:
value = test.loc[0][1:]
print dict( value )

{'stalk_root': 0, 'stalk_color_above_ring': 7, 'stalk_surface_above_ring': 2, 'gill_attachment': 1, 'gill_spacing': 1, 'habitat': 1, 'stalk_shape': 0, 'spore_print_color': 7, 'gill_size': 0, 'gill_color': 2, 'ring_number': 2, 'veil_type': 0, 'odor': 5, 'stalk_color_below_ring': 7, 'cap_surface': 0, 'ring_type': 4, 'population': 3, 'veil_color': 2, 'stalk_surface_below_ring': 2, 'bruises': 0, 'cap_color': 8, 'cap_shape': 5}


In [68]:
accuray = ( np.array( preds ) == test['type'] ).sum() / float( test.shape[0] )
print accuray

ValueError: Lengths must match to compare